We extract main news using element 'h3' and sub news using 'h4' as of now. There is a common method for storing the
headlines-fetched data to pandas DataFrame. <br/>
Before storing, we search for certain keywords.These are 'surge', 'acquisitions', 'IPO', and 'initial public offering'.<br/>
We fetch headlines using soup and html elements and create a list of headlines. Then browse those headlines, feed them to<br/>
newspaper API available here: http://newspaper-demo.herokuapp.com/<br/>
This API returns a table containing information like news title, keywords, summary, author etc. It doesn't work perfectly
but still is useful.<br/>
Finally we store this scraped data for all the headlines to a pandas DataFrame.

In [82]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

This method searches for keyword 'surge' and return a list of sentences containing the word. Otherwise, it returns None.

In [1]:
def keyword_surge(news_text):
    count = 0
    matching_sentences = []
    list_of_sentences = news_text.split('. ') #Sentence splitter as new sentence starts after a full stop followed by space
    for sentence in list_of_sentences:
        if 'surge' in sentence:
            count += 1
            matching_sentences.append(sentence)
    if count == 0:
        return None
    return matching_sentences

Method for searching the keyword 'acquisitions'

In [111]:
def keyword_acquisitions(news_text):
    count = 0
    matching_sentences = []
    list_of_sentences = news_text.split('. ')
    for sentence in list_of_sentences:
        if 'acquisitions' in sentence:
            count += 1
            matching_sentences.append(sentence)
    if count == 0:
        return None
    return matching_sentences

In [3]:
def keyword_ipo(news_text):
    count = 0
    matching_sentences = []
    list_of_sentences = news_text.split('. ')
    for sentence in list_of_sentences:
        if 'IPO' in sentence or 'initial public offering' in sentence:
            count += 1
            matching_sentences.append(sentence)
    if count == 0:
        return None
    return matching_sentences

The below method accepts a url, and makes a requests query to fetch it.

In [67]:
def page_fetch(url):
    page_fetch = requests.get(url)
    return page_fetch 

This method makes use of the above method, takes the fetched page, and scraps its content using the Soup call. It returns the soup object.

In [71]:
def get_soup(url):
    page_fetch = requests.get(url)
    soup = BeautifulSoup(page_fetch.content, 'html.parser')
    return soup

This method returns a list of headline urls. It does so by accepting an element and feeding it to the soup object.

In [2]:
def get_headlines(element): #Element is the html element like 'h3' to scrap specific content using the soup object
    soup = get_soup('https://news.google.com/') 
    section = soup.find_all(element)
    headlines = [] #List to hold headline urls
    for tag in section: #For all the elements (eg., 'h3')returned by the soup object
        for i in tag.find_all():
            headlines.append('https://news.google.com{}'.format(i['href'][1:])) #Extracting href from the headlines and
                                                                                #concatenating them to parent url "https://news.google.com"
    return headlines

A global DataFrame to store the scraped news.

In [182]:
df = pd.DataFrame(columns=['url', 'title', 'keywords', 'datetime'])

This method accepts an html element ('h3' for main news and 'h4' for sub news), creates a headline list, visits those headlines,<br/>
and feeds them to the newspaper API and stores the output data to the DataFrame.

In [183]:
def store_news(element):
    global df #Global DatFrame
    title = keywords = ''
    keyword_matching_sentences = []
    count = 0
    for headline in get_headlines(element): #Visiting the list of headlines
        count += 1
        soup = get_soup('http://newspaper-demo.herokuapp.com/articles/show?url_to_clean={}'.format(headline)) # Feeding the heading to newspaper API
        #Newspaper API result is a table which needs to be scraped
        table = soup.find('table') 
        table_body = table.find('tbody') # Entering table body
        rows = table_body.find_all('tr') # Table rows
        for row in rows:  # For each row
            columns = row.find_all('td') # Columns for each row
        # There are 2 columns in each row, 1st for example is 'Title', and the 2nd, is the title itself
            if columns[0].text.strip() == 'Title': # text.strip() removes the tags as column[0] is <td>Title</td> and we need 'Title' only
                title = columns[1].text.strip()
            if columns[0].text.strip() == 'Keywords (NLP)':
                keywords = columns[1].text.strip()

            if columns[0].text.strip() == 'Text':
            '''We have to first search for keywords like 'surge', 'IPO' etc. We already have 3 methods for them.
               We are trying to print in a detailed manner. For example,
               
               News 1: "Corona Virus spreads...."
               Term 'surge' occurs in the following sentences of this news: [sentence this, sentence this]
               Sorry! Term 'surge' couldn't be found in the news!
               
               News 2: "Corona Virus spreads...."
               Term 'surge' occurs in the following sentences of this news: [sentence this, sentence this]
               Sorry! Term 'surge' couldn't be found in the news!
            '''
            '''
               Some news don't have title in results, for them we only print like
               News 1
               Term 'surge' occurs in the following sentences of this news: [sentence this, sentence this]
               Sorry! Term 'surge' couldn't be found in the news!
            '''
                if title:
                    print('News ', count, ': ', title) 
                else:
                    print('News ', count)
            # Extract the text of the news from the API result
                text = columns[1].text.strip()

                keyword_matching_sentences = keyword_surge(text) # Invoking the keyword searching methods

                if keyword_matching_sentences:
                    print("-> Term 'surge' occurs in the following sentences of this news <-")
                    print(matching_sentences)
                else:
                    print("-> Sorry! Term 'surge' couldn't be found in the news! <-")

                keyword_matching_sentences = keyword_acquisitions(text)

                if keyword_matching_sentences:
                    print("-> Term 'acquisitions' occurs in the following sentences of this news <-")
                    print(keyword_matching_sentences)
                else:
                    print("-> Sorry! Term 'acquisitions' couldn't be found in the news! <-")

                keyword_matching_sentences = keyword_ipo(text)

                if keyword_matching_sentences:
                    print("-> Term 'ipo' or 'initial public offering' occurs in the following sentences of this news <-")
                    print(keyword_matching_sentences)
                else:
                    print("-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-")
                print("------------------------------------------------------------------")

                # Appending the result to the DataFrame
                if text != None  and title != None and keywords != None:  
                        df = df.append(
                            {
                                'url' : headline,
                                'title' : title,
                                'keywords' : keywords,
                                'datetime' : datetime.datetime.now()
                            } , ignore_index=True)

Invoking the above method for main news

In [184]:
store_news('h3')

News  1 :  PM Narendra Modi Condemns Harassment Of Doctors, Says Those In White Coats Form Of God
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  2 :  Coronavirus: Tamil Nadu records 5 new cases and first death, total 23 COVID-19 patients in state
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  3 :  India condemns Gurdwara attack, promises all possible help to Sikhs and Hindus of Afghanistan
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' 

News  24 :  Kartik Aaryan's ROFL "21 Din Mein Paise Double" Meme From Akshay Kumar's Phir Hera Pheri
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  25 :  Big B Pens Poem Urging Citizens to Cooperate During 21-day Lockdown
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  26 :  Arjun Kapoor pens emotional post for mom Mona on death anniversary: ‘My world was shattered when you left’
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial

Our DataFrame

In [186]:
df

,url,title,keywords,datetime
0,https://news.google.com/articles/CAIiENySuYCgr...,PM Narendra Modi Condemns Harassment Of Doctor...,,2020-03-25 19:05:30.956410
1,https://news.google.com/articles/CBMilgFodHRwc...,Coronavirus: Tamil Nadu records 5 new cases an...,"god, coronavirus, narendra, prime, doctors, st...",2020-03-25 19:05:32.043537
2,https://news.google.com/articles/CAIiEHNEO0wz1...,"India condemns Gurdwara attack, promises all p...","coronavirus, minister, vijayabaskar, state, ta...",2020-03-25 19:05:33.656190
3,https://news.google.com/articles/CAIiENQpULdrJ...,Tamil Nadu records first COVID-19 death as 54-...,"afghanistan, sikhs, community, promises, minor...",2020-03-25 19:05:37.337931
4,https://news.google.com/articles/CAIiEGXCCync_...,Coronavirus Outbreak: NPR updation exercise de...,"minister, patient, history, treatment, tamil, ...",2020-03-25 19:05:38.741056
5,https://news.google.com/articles/CBMikAFodHRwc...,India’s coronavirus curve flattens slightly as...,"coronavirus, narendra, 21day, corona, npr, ban...",2020-03-25 19:05:39.719905
6,https://news.google.com/articles/CAIiEB6IrS-HP...,277 people evacuated from coronavirus-hit Iran...,"coronavirus, confirmed, far, cases, deaths, lo...",2020-03-25 19:05:41.276880
7,https://news.google.com/articles/CAIiEOgZSZ2rW...,4.5 Lakh May Need ICU Admissions in Delhi Alon...,"army, rajasthan, iran, arrived, 277, reach, me...",2020-03-25 19:05:42.836114
8,https://news.google.com/articles/CAIiENP6yj3Yk...,"15,000 Ltrs Milk, 10,000 kg Veg Dumped","cases, icmr, days, infected, study, infections...",2020-03-25 19:05:44.009865
9,https://news.google.com/articles/CAIiEMyxBie8x...,"No final exams, all pass: TN, Puducherry tell ...","ltrs, kg, mr, 15000, essential, dumped, delive...",2020-03-25 19:05:47.527482


Now, invoking that same method for sub news

In [187]:
store_news('h4')

News  1 :  Modi's unprecedented gamble will change India irrevocably
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  2 :  Transfer cash in bank accounts of poor during lockdown
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  3 :  Shutting Down 1.3 Billion For 21 Days A Huge Gamble For PM Narendra Modi
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
-----------------------------

News  24 :  277 Indians Evacuated From Iran Shifted To Army Facility In Jodhpur
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  25 :  ICMR: Isolation may cut peak coronavirus numbers by 89%
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  26 :  Don't use hydroxychloroquine medicine for coronavirus without prescription
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
-------------

News  47 :  Modi govt's Rs 1.5 lakh crore warchest to defeat coronavirus economic slowdown; mega package likely this week
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  48
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  49 :  financial package: Stimulus package likely by end of this week
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------

News  70 :  EMUI 10 is now installed on 100 million devices around the globe
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  71 :  Redmi K20 series sales exceed 5 million, 50% to new users
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  72 :  Xiaomi launches Redmi SmartTV Max with a gigantic 98-inch display
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
-----------------------

News  95 :  Tokyo Olympics postponed to 2021 amid coronavirus crisis
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  96
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  97 :  Tokyo Olympics Postponement Necessary for Athletes' Well Being
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  98 :  ‘Focusing on fit

News  122
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  123 :  Hantavirus in China: 5 things you need to know about the virus - ​Another virus?
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  124
-> Sorry! Term 'surge' couldn't be found in the news! <-
-> Sorry! Term 'acquisitions' couldn't be found in the news! <-
-> Sorry! Terms 'ipo' or 'initial public offering' couldn't be found in the news! <-
------------------------------------------------------------------
News  125
-> Sorry! Term 'surge' couldn't be found in the news! <-


DataFrame

In [188]:
df

,url,title,keywords,datetime
0,https://news.google.com/articles/CAIiENySuYCgr...,PM Narendra Modi Condemns Harassment Of Doctor...,,2020-03-25 19:05:30.956410
1,https://news.google.com/articles/CBMilgFodHRwc...,Coronavirus: Tamil Nadu records 5 new cases an...,"god, coronavirus, narendra, prime, doctors, st...",2020-03-25 19:05:32.043537
2,https://news.google.com/articles/CAIiEHNEO0wz1...,"India condemns Gurdwara attack, promises all p...","coronavirus, minister, vijayabaskar, state, ta...",2020-03-25 19:05:33.656190
3,https://news.google.com/articles/CAIiENQpULdrJ...,Tamil Nadu records first COVID-19 death as 54-...,"afghanistan, sikhs, community, promises, minor...",2020-03-25 19:05:37.337931
4,https://news.google.com/articles/CAIiEGXCCync_...,Coronavirus Outbreak: NPR updation exercise de...,"minister, patient, history, treatment, tamil, ...",2020-03-25 19:05:38.741056
5,https://news.google.com/articles/CBMikAFodHRwc...,India’s coronavirus curve flattens slightly as...,"coronavirus, narendra, 21day, corona, npr, ban...",2020-03-25 19:05:39.719905
6,https://news.google.com/articles/CAIiEB6IrS-HP...,277 people evacuated from coronavirus-hit Iran...,"coronavirus, confirmed, far, cases, deaths, lo...",2020-03-25 19:05:41.276880
7,https://news.google.com/articles/CAIiEOgZSZ2rW...,4.5 Lakh May Need ICU Admissions in Delhi Alon...,"army, rajasthan, iran, arrived, 277, reach, me...",2020-03-25 19:05:42.836114
8,https://news.google.com/articles/CAIiENP6yj3Yk...,"15,000 Ltrs Milk, 10,000 kg Veg Dumped","cases, icmr, days, infected, study, infections...",2020-03-25 19:05:44.009865
9,https://news.google.com/articles/CAIiEMyxBie8x...,"No final exams, all pass: TN, Puducherry tell ...","ltrs, kg, mr, 15000, essential, dumped, delive...",2020-03-25 19:05:47.527482


We observe that the newspaper API isn't completely reliable as not for every news, we have its title, summary, or keywords
scraped.<br/>But then, as there are different news websites to be scraped, it takes more time to develop individual scrapers although that is certainly a <br/>reliable improvement over this work.